# Bayesian optimisation with known observational noise

In this example, NUBO is used for sequential single-point optimisation for situations where the observational noise from taking the measurements is known. We assume that the variance of the observational noise of our black box funcion (simulated here by the `Hartmann6D` function) to be equal to $\sigma^2 = 0.025$. The Bayesian optimisation loop compared to the case with unknown noise differs only in terms of the likelihood that we use. Here, we use the `FixedNoiseGaussianLikelihood` and specify the observational noise variance for each data point (for this example, the same variance is used for all points). We also allow the likelihood to estimate any additional noise. The optimisation loop is run for 40 iterations and finds a solution close the true optimum of -3.3224.

In [1]:
import torch
from nubo.acquisition import ExpectedImprovement, UpperConfidenceBound
from nubo.models import GaussianProcess, fit_gp
from nubo.optimisation import lbfgsb
from nubo.test_functions import Hartmann6D
from nubo.utils import gen_inputs
from gpytorch.likelihoods import FixedNoiseGaussianLikelihood


# test function
func = Hartmann6D(minimise=False, noise_std=0.025)
dims = func.dims
bounds = func.bounds

# training data
x_train = gen_inputs(num_points=dims*5,
                     num_dims=dims,
                     bounds=bounds)
y_train = func(x_train)

# Bayesian optimisation loop
iters = 40

for iter in range(iters):
    
    # specify Gaussian process
    likelihood = FixedNoiseGaussianLikelihood(noise=torch.ones(x_train.size(0))*0.025, learn_additional_noise=True)
    gp = GaussianProcess(x_train, y_train, likelihood=likelihood)
    
    # fit Gaussian process
    fit_gp(x_train, y_train, gp=gp, likelihood=likelihood, lr=0.1, steps=200)

    # specify acquisition function
    # acq = ExpectedImprovement(gp=gp, y_best=torch.max(y_train))
    acq = UpperConfidenceBound(gp=gp, beta=1.96**2)

    # optimise acquisition function
    x_new, _ = lbfgsb(func=acq, bounds=bounds, num_starts=5)

    # evaluate new point
    y_new = func(x_new)
    
    # add to data
    x_train = torch.vstack((x_train, x_new))
    y_train = torch.hstack((y_train, y_new))

    # print new best
    if y_new > torch.max(y_train[:-1]):
        print(f"New best at evaluation {len(y_train)}: \t Inputs: {x_new.numpy().reshape(dims).round(4)}, \t Outputs: {-y_new.numpy().round(4)}")

# results
best_iter = int(torch.argmax(y_train))
print(f"Evaluation: {best_iter+1} \t Solution: {-float(y_train[best_iter]):.4f}")


New best at evaluation 35: 	 Inputs: [0.1645 0.2206 0.3035 0.2072 0.194  0.869 ], 	 Outputs: [-1.6695]
New best at evaluation 36: 	 Inputs: [0.4713 0.7459 1.     0.6095 0.5726 0.    ], 	 Outputs: [-2.3638]
New best at evaluation 42: 	 Inputs: [0.1779 0.0974 0.3987 0.2433 0.2859 0.7401], 	 Outputs: [-2.9697]
New best at evaluation 44: 	 Inputs: [0.414  0.8375 1.     0.5714 0.5749 0.    ], 	 Outputs: [-2.9842]
New best at evaluation 45: 	 Inputs: [0.2113 0.0234 0.4707 0.2358 0.286  0.7096], 	 Outputs: [-3.0098]
New best at evaluation 52: 	 Inputs: [0.3824 0.8989 0.7951 0.5489 0.5283 0.    ], 	 Outputs: [-3.0416]
New best at evaluation 57: 	 Inputs: [0.2381 0.1042 0.4973 0.2776 0.2888 0.6892], 	 Outputs: [-3.2606]
New best at evaluation 59: 	 Inputs: [0.2256 0.0998 0.4559 0.2737 0.2983 0.6449], 	 Outputs: [-3.297]
Evaluation: 59 	 Solution: -3.2970
